<a href="https://colab.research.google.com/github/savindumahasen/USAAirLineSentimentAnalysis/blob/main/USAirLinesSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
## import the pandas data library

In [179]:
import pandas as pd

In [180]:
df=pd.read_csv("Tweets.csv")
df.head(5)
df.shape

(14640, 15)

In [181]:
df =df[['airline_sentiment','text']]
df.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [182]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @AmericanAir that's 16+ extra hours of travel ...    2  
neutral                                           @SouthwestAir sent    5  
positive                                            @JetBlue thanks!    5

## Preprocessing text

In [183]:
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()
text="@VirginAmerica What @dhepburn said."

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [184]:
df['cleaned_text']= df['text'].apply(clean_text)

In [185]:
df.head(5)


,airline_sentiment,text,cleaned_text
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .
1,positive,@VirginAmerica plus you've added commercials t...,@ virginamerica plu 've ad commerci experi ......
2,neutral,@VirginAmerica I didn't today... Must mean I n...,@ virginamerica n't today ... must mean need t...
3,negative,@VirginAmerica it's really aggressive to blast...,@ virginamerica 's realli aggress blast obnoxi...
4,negative,@VirginAmerica and it's a really big bad thing...,@ virginamerica 's realli big bad thing


## Feature Extraction

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec= TfidfVectorizer(max_features=3000)
X= tf_vec.fit_transform(df['cleaned_text']).toarray()
X.shape


(14640, 3000)

In [187]:
Y = df['airline_sentiment'].copy()
Y.head(5)

0     neutral
1    positive
2     neutral
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [188]:
X.shape

(14640, 3000)

In [189]:
Y.shape

(14640,)

In [190]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=2)

In [191]:
X_train.shape

(11712, 3000)

In [192]:
y_train.shape

(11712,)

In [193]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [194]:
from sklearn.metrics import  accuracy_score
y_pred= model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7219945355191257


In [195]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test,y_pred))



0.7506830601092896


In [196]:
sample_data = ["@ virginamerica @ dhepburn said ."]

# Transform the sample data using the loaded vectorizer
X_sample = tf_vec.transform(sample_data).toarray()

# Predict using the loaded model
predictions = model.predict(X_sample)
predictions[0]

if predictions[0]=='negative':
  print("negative")
elif predictions[0]=='positive':
  print("postive")
else:
  print("neutral")


# Print the results
#for sample, prediction in zip(sample_data, predictions):
 #   sentiment = "positive" if prediction == 'positive' else "negative" if prediction == 2 else "neutral"
  #  print(f"Sample: {sample}\nPredicted Sentiment: {sentiment}\n")


neutral
